<a href="https://colab.research.google.com/github/signalcompose/MusicTransformer-tensorflow2.0/blob/master/GoogleCoraboratory/MusicTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###前処理
* Music Transformer のリポジトリダウンロード
* リポジトリのサブモジュールをダウンロード
* [将来削除予定]経過時間を表示するブランチにチェックアウトする
* Yamaha e-piano competition のデータダウンロード
* preprocess に必要なモジュールのインストール
* preprocess.py の実行

In [16]:
!git clone https://github.com/signalcompose/MusicTransformer-tensorflow2.0.git 
%cd MusicTransformer-tensorflow2.0/
!git submodule init && git submodule update
!git checkout -b feature/PrintTrainTimer origin/feature/PrintTrainTimer
!mkdir -p dataset/ecomp/midi
!sh dataset/scripts/ecomp_piano_downloader.sh dataset/ecomp/midi
!mkdir -p dataset/ecomp/processed
!pip install progress pretty_midi config
!python preprocess.py dataset/ecomp/midi dataset/ecomp/processed

ストリーミング出力は最後の 5000 行に切り捨てられました。

Levitsky03.MID      100%[===================>]  83.97K  --.-KB/s    in 0s      

2020-06-04 09:58:20 (323 MB/s) - ‘dataset/ecomp/midi/Levitsky03.MID’ saved [85986/85986]

--2020-06-04 09:58:20--  http://www.piano-e-competition.com/MIDIFiles/2008/Levitsky04.MID
Reusing existing connection to www.piano-e-competition.com:80.
HTTP request sent, awaiting response... 200 OK
Length: 20506 (20K) [audio/mid]
Saving to: ‘dataset/ecomp/midi/Levitsky04.MID’

Levitsky04.MID      100%[===================>]  20.03K  --.-KB/s    in 0s      

2020-06-04 09:58:20 (250 MB/s) - ‘dataset/ecomp/midi/Levitsky04.MID’ saved [20506/20506]

--2020-06-04 09:58:20--  http://www.piano-e-competition.com/MIDIFiles/2008/Levitsky05.MID
Reusing existing connection to www.piano-e-competition.com:80.
HTTP request sent, awaiting response... 200 OK
Length: 38987 (38K) [audio/mid]
Saving to: ‘dataset/ecomp/midi/Levitsky05.MID’

Levitsky05.MID      100%[===================>]  38.07K  --.-KB/s 

## トレーニング
